In [8]:
#SET UP:

In [1]:
print("HELLO")

HELLO


In [2]:
print("hi")

hi


In [3]:
from pathlib import Path
import openmeteo_requests
from dotenv import load_dotenv
import datetime
import hopsworks
import pandas as pd

HOPSWORKS_HOST="c.app.hopsworks.ai"
#May need https://
HOPSWORKS_PROJECT="AirQualityKTHLab1"
city = "stockholm"
pollen_types = ["alder_pollen","birch_pollen","grass_pollen" ,"mugwort_pollen" ,"olive_pollen" ,"ragweed_pollen"]
#Stockholm: 59.334591, 18.063240
#new: 59.338295,18.055603
latitude =59.338295
longitude = 18.055603

today = datetime.date.today()

root_dir = Path.cwd()
print(root_dir)


load_dotenv()

project = hopsworks.login(host=HOPSWORKS_HOST, project=HOPSWORKS_PROJECT)


/home/viktor/Pollen-Predictor/notebooks
2026-01-02 16:49:18,694 INFO: Initializing external client
2026-01-02 16:49:18,695 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-02 16:49:21,050 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286344


In [4]:
#Featching historical data!
def fetch_historical_wether(latitude, longitude, date_start, date_end):
    # https://historical-forecast-api.open-meteo.com/v1/forecast?latitude=52.52&longitude=13.41&start_date=2025-12-16&end_date=2025-12-30&daily=temperature_2m_mean,precipitation_sum,dew_point_2m_mean,wind_speed_10m_mean,sunshine_duration&timezone=Europe%2FBerlin
    open_meteo = openmeteo_requests.Client()
    
    
    daily_variables = ["temperature_2m_mean", "precipitation_sum", "dew_point_2m_mean", "wind_speed_10m_mean", "sunshine_duration"]
    params ={
        "timezone": "Europe/Berlin",
        "start_date": str(date_start),
        "end_date": str(date_end),
        "latitude": latitude,
        "longitude": longitude,
        "daily": daily_variables,
    }

    url = "https://historical-forecast-api.open-meteo.com/v1/forecast"


    response = open_meteo.weather_api(url, params=params)[0]
    
    #print("utc_offset_seconds: " , response.utc_offset_seconds)

    daily= response.Daily()

    #Unpacking the variables similarly as to how we did in Lab1 (from https://github.com/featurestorebook/mlfs-book/)
    daily_time = pd.date_range(start =pd.to_datetime(daily.Time(), unit = "s", utc=True),
        end = pd.to_datetime(daily.TimeEnd(),unit = "s", utc=True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )
    #But according to https://github.com/open-meteo/sdk?tab=readme-ov-file#weatherapiresponse, time is in GMT:

    daily_time = daily_time.tz_convert("Europe/Stockholm")

    data = {
        "date": daily_time,
        "temperature_2m_mean": daily.Variables(0).ValuesAsNumpy(),
        "precipitation_sum": daily.Variables(1).ValuesAsNumpy(),
        "dew_point_2m_mean": daily.Variables(2).ValuesAsNumpy(),
        "wind_speed_10m_mean": daily.Variables(3).ValuesAsNumpy(),
        "sunshine_duration": daily.Variables(4).ValuesAsNumpy(),
    }

    df = pd.DataFrame(data)
    df["date"] =df["date"].dt.tz_localize(None).dt.normalize() #converting to calender date!!
    
    return df


In [5]:
#to fill in the gap:
def fetch_recent_historical_wether_forecasts(latitude, longitude, date_start, date_end):
    open_meteo = openmeteo_requests.Client()
    
    
    daily_variables = ["temperature_2m_mean", "precipitation_sum", "dew_point_2m_mean", "wind_speed_10m_mean", "sunshine_duration"]
    params ={
        "timezone": "Europe/Berlin",
        "start_date": str(date_start),
        "end_date": str(date_end),
        "latitude": latitude,
        "longitude": longitude,
        "daily": daily_variables,
    }

    url = "https://api.open-meteo.com/v1/forecast"


    response = open_meteo.weather_api(url, params=params)[0]

    daily= response.Daily()

    #Unpacking the variables similarly as to how we did in Lab1 (from https://github.com/featurestorebook/mlfs-book/)
    daily_time = pd.date_range(start =pd.to_datetime(daily.Time(), unit = "s", utc=True),
        end = pd.to_datetime(daily.TimeEnd(),unit = "s", utc=True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )
    daily_time = daily_time.tz_convert("Europe/Stockholm")

    data = {
        "date": daily_time,
        "temperature_2m_mean": daily.Variables(0).ValuesAsNumpy(),
        "precipitation_sum": daily.Variables(1).ValuesAsNumpy(),
        "dew_point_2m_mean": daily.Variables(2).ValuesAsNumpy(),
        "wind_speed_10m_mean": daily.Variables(3).ValuesAsNumpy(),
        "sunshine_duration": daily.Variables(4).ValuesAsNumpy(),
    }

    df = pd.DataFrame(data)
    df["date"] =df["date"].dt.tz_localize(None).dt.normalize()
    return df
    

In [6]:
def get_full_wether_history(latitude, longitude, date_start, date_end):

    df_wether_hist = fetch_historical_wether(latitude, longitude, date_start, date_end)

    last_recorded_date = df_wether_hist["date"].max().date()

    if last_recorded_date >= date_end: 
        return df_wether_hist.sort_values("date").reset_index(drop=True)

    fill_start =last_recorded_date + datetime.timedelta(days=1) #But this needs to be a string i think!

    df_wether_recent= fetch_recent_historical_wether_forecasts(latitude, longitude, str(fill_start), date_end)

    df_wether_concat = pd.concat([df_wether_hist,df_wether_recent ], ignore_index=True)

    df_wether = df_wether_concat.sort_values("date").drop_duplicates(subset="date").reset_index(drop=True)

    return df_wether
    

In [7]:
def fetch_historical_pollen(latitude, longitude, date_start, date_end):
    #https://air-quality-api.open-meteo.com/v1/air-quality?latitude=52.52&longitude=13.41&hourly=alder_pollen,birch_pollen,grass_pollen,mugwort_pollen,olive_pollen,ragweed_pollen&timezone=Europe%2FBerlin&start_date=2022-12-25&end_date=2026-01-06

    open_meteo = openmeteo_requests.Client()
    
    params ={
        "timezone": "Europe/Berlin",
        "start_date": str(date_start),
        "end_date": str(date_end),
        "latitude": latitude,
        "longitude": longitude,
        "hourly": pollen_types,
    }

    url = "https://air-quality-api.open-meteo.com/v1/air-quality"


    response = open_meteo.weather_api(url, params=params)[0]

    hourly= response.Hourly()
    #n_steps = len(hourly.Variables(0).ValuesAsNumpy())

    #Unpacking the variables similarly as to how we did in Lab1 (from https://github.com/featurestorebook/mlfs-book/)
    time_hourly = pd.date_range(start =pd.to_datetime(hourly.Time(), unit ="s", utc=True),
        end = pd.to_datetime(hourly.TimeEnd(),unit = "s", utc=True),
        #periods = n_steps,
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )
    time_hourly = time_hourly.tz_convert("Europe/Stockholm")

    data = {
        "date_time": time_hourly,
    }
    #unpacking all the data:
    for i, var in enumerate(pollen_types):
        data[var] = hourly.Variables(i).ValuesAsNumpy()

    df = pd.DataFrame(data)
    
    df["date"] = df["date_time"].dt.tz_localize(None).dt.normalize()

    df_daily_max  = df.groupby("date")[pollen_types].max().reset_index()
    df_daily_max = df_daily_max.sort_values("date").reset_index(drop=True)
    wanted_columns = ["date"] + pollen_types

    return df_daily_max[wanted_columns]

""" def fetch_historical_pollen(latitude, longitude, date_start, date_end):
    #https://air-quality-api.open-meteo.com/v1/air-quality?latitude=52.52&longitude=13.41&hourly=alder_pollen,birch_pollen,grass_pollen,mugwort_pollen,olive_pollen,ragweed_pollen&timezone=Europe%2FBerlin&start_date=2022-12-25&end_date=2026-01-06

    open_meteo = openmeteo_requests.Client()
    
    params ={
        "timezone": "Europe/Berlin",
        "start_date": str(date_start),
        "end_date": str(date_end),
        "latitude": latitude,
        "longitude": longitude,
        "hourly": pollen_types,
    }

    url = "https://air-quality-api.open-meteo.com/v1/air-quality"


    response = open_meteo.weather_api(url, params=params)[0]

    hourly= response.Hourly()
    #n_steps = len(hourly.Variables(0).ValuesAsNumpy())

    #Unpacking the variables similarly as to how we did in Lab1 (from https://github.com/featurestorebook/mlfs-book/)
    time_hourly = pd.date_range(start =pd.to_datetime(hourly.Time(), unit ="s"),
        end = pd.to_datetime(hourly.TimeEnd(),unit = "s"),
        #periods = n_steps,
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )


    data = {
        "date_time": time_hourly,
    }
    #unpacking all the data:
    for i, var in enumerate(pollen_types):
        data[var] = hourly.Variables(i).ValuesAsNumpy()

    df = pd.DataFrame(data)
    
    df["date"] = df["date_time"].dt.tz_localize(None).dt.normalize()

    df_daily_max  = df.groupby("date")[pollen_types].max().reset_index()
    df_daily_max = df_daily_max.sort_values("date").reset_index(drop=True)
    wanted_columns = ["date"] + pollen_types

    return df_daily_max[wanted_columns] """


' def fetch_historical_pollen(latitude, longitude, date_start, date_end):\n    #https://air-quality-api.open-meteo.com/v1/air-quality?latitude=52.52&longitude=13.41&hourly=alder_pollen,birch_pollen,grass_pollen,mugwort_pollen,olive_pollen,ragweed_pollen&timezone=Europe%2FBerlin&start_date=2022-12-25&end_date=2026-01-06\n\n    open_meteo = openmeteo_requests.Client()\n    \n    params ={\n        "timezone": "Europe/Berlin",\n        "start_date": str(date_start),\n        "end_date": str(date_end),\n        "latitude": latitude,\n        "longitude": longitude,\n        "hourly": pollen_types,\n    }\n\n    url = "https://air-quality-api.open-meteo.com/v1/air-quality"\n\n\n    response = open_meteo.weather_api(url, params=params)[0]\n\n    hourly= response.Hourly()\n    #n_steps = len(hourly.Variables(0).ValuesAsNumpy())\n\n    #Unpacking the variables similarly as to how we did in Lab1 (from https://github.com/featurestorebook/mlfs-book/)\n    time_hourly = pd.date_range(start =pd.to_

In [8]:
#2024-05-18
#2021-01-01 #seems sort of reasnoable

date_start = "2021-01-01"

df_wether = get_full_wether_history(latitude, longitude, date_start, date_end=today)
df_wether = df_wether.sort_values("date").reset_index(drop=True)
df_wether

,date,temperature_2m_mean,precipitation_sum,dew_point_2m_mean,wind_speed_10m_mean,sunshine_duration
0,2021-01-01,0.162500,3.000000,0.072917,5.461522,0.000000
1,2021-01-02,0.887500,2.300000,0.793750,7.108414,0.000000
2,2021-01-03,-0.816667,0.000000,-1.997917,10.058395,0.000000
3,2021-01-04,0.193750,0.900000,-1.656250,11.215699,0.000000
4,2021-01-05,-0.308333,0.000000,-2.522917,11.719703,611.854492
...,...,...,...,...,...,...
1823,2025-12-29,-0.093417,1.000000,-6.607390,27.270002,13654.375000
1824,2025-12-30,-2.537167,0.200000,-8.133086,26.804996,3635.480957
1825,2025-12-31,-4.160083,0.500000,-5.608835,7.575000,13690.069336
1826,2026-01-01,-0.195500,11.000001,-1.576530,15.855001,0.000000


In [10]:
#Feature engineering:
df_wether["doy"] = df_wether["date"].dt.dayofyear

#temp:
df_wether["temp_7d_mean"] = df_wether["temperature_2m_mean"].rolling(window=7,min_periods=1).mean()
df_wether["temp_14d_mean"] = df_wether["temperature_2m_mean"].rolling(window=14, min_periods=1).mean()

#percipitaion:
df_wether["precip_3d_sum"] = df_wether["precipitation_sum"].rolling(window=3, min_periods=1).sum()
df_wether["precip_7d_sum"] = df_wether["precipitation_sum"].rolling(window=7, min_periods=1).sum()

df_wether["dew_point_3d_mean"] = df_wether["dew_point_2m_mean"].rolling(window=3, min_periods=1).mean()

df_wether

,date,temperature_2m_mean,precipitation_sum,dew_point_2m_mean,wind_speed_10m_mean,sunshine_duration,doy,temp_7d_mean,temp_14d_mean,precip_3d_sum,precip_7d_sum,dew_point_3d_mean
0,2021-01-01,0.162500,3.000000,0.072917,5.461522,0.000000,1,0.162500,0.162500,3.000000,3.000000,0.072917
1,2021-01-02,0.887500,2.300000,0.793750,7.108414,0.000000,2,0.525000,0.525000,5.300000,5.300000,0.433333
2,2021-01-03,-0.816667,0.000000,-1.997917,10.058395,0.000000,3,0.077778,0.077778,5.300000,5.300000,-0.377083
3,2021-01-04,0.193750,0.900000,-1.656250,11.215699,0.000000,4,0.106771,0.106771,3.200000,6.200000,-0.953472
4,2021-01-05,-0.308333,0.000000,-2.522917,11.719703,611.854492,5,0.023750,0.023750,0.900000,6.200000,-2.059028
...,...,...,...,...,...,...,...,...,...,...,...,...
1823,2025-12-29,-0.093417,1.000000,-6.607390,27.270002,13654.375000,363,0.960155,3.255244,1.000000,1.000000,-4.563624
1824,2025-12-30,-2.537167,0.200000,-8.133086,26.804996,3635.480957,364,0.649143,2.549738,1.200000,1.200000,-6.248264
1825,2025-12-31,-4.160083,0.500000,-5.608835,7.575000,13690.069336,365,0.266107,1.785899,1.700000,1.700000,-6.783104
1826,2026-01-01,-0.195500,11.000001,-1.576530,15.855001,0.000000,1,0.120571,1.374292,11.700001,12.700001,-5.106150


In [11]:
df_pollen = fetch_historical_pollen(latitude, longitude, date_start, date_end=today)
df_pollen = df_pollen.sort_values("date").reset_index(drop=True)
df_pollen

,date,alder_pollen,birch_pollen,grass_pollen,mugwort_pollen,olive_pollen,ragweed_pollen
0,2021-01-01,0.0,0.0,0.0,NaN,0.0,0.0
1,2021-01-02,0.0,0.0,0.0,NaN,0.0,0.0
2,2021-01-03,0.0,0.0,0.0,NaN,0.0,0.0
3,2021-01-04,0.0,0.0,0.0,NaN,0.0,0.0
4,2021-01-05,0.0,0.0,0.0,NaN,0.0,0.0
...,...,...,...,...,...,...,...
1823,2025-12-29,0.0,0.0,0.0,0.0,0.0,0.0
1824,2025-12-30,0.0,0.0,0.0,0.0,0.0,0.0
1825,2025-12-31,0.0,0.0,0.0,0.0,0.0,0.0
1826,2026-01-01,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
print(df_pollen[147:170])

          date  alder_pollen  birch_pollen  grass_pollen  mugwort_pollen  \
147 2021-05-28           0.1     72.099998      1.700000             NaN   
148 2021-05-29           0.2    243.300003      1.300000             NaN   
149 2021-05-30           0.1    201.399994      1.800000             NaN   
150 2021-05-31           3.1    478.100006      3.200000             NaN   
151 2021-06-01           0.6    260.600006      1.700000             NaN   
152 2021-06-02           0.1     96.000000      3.700000             NaN   
153 2021-06-03           0.1     44.599998      2.300000             NaN   
154 2021-06-04           0.0     16.000000      4.700000             NaN   
155 2021-06-05           0.0      9.900000      4.700000             NaN   
156 2021-06-06           0.0     14.500000      4.400000             NaN   
157 2021-06-07           0.0     13.200000      3.800000             NaN   
158 2021-06-08           0.0      0.600000      3.200000             NaN   
159 2021-06-

#Hopsworks

In [13]:
for pollen_type in pollen_types:

    lag1 = df_pollen[pollen_type].shift(1)
    lag2 = df_pollen[pollen_type].shift(2)
    lag3 = df_pollen[pollen_type].shift(3)

    df_pollen[f"{pollen_type}_lag1"] = lag1.ffill()
    df_pollen[f"{pollen_type}_lag2"] = lag2.ffill()
    df_pollen[f"{pollen_type}_lag3"] = lag3.ffill()

In [14]:
print(df_pollen[147:170])

          date  alder_pollen  birch_pollen  grass_pollen  mugwort_pollen  \
147 2021-05-28           0.1     72.099998      1.700000             NaN   
148 2021-05-29           0.2    243.300003      1.300000             NaN   
149 2021-05-30           0.1    201.399994      1.800000             NaN   
150 2021-05-31           3.1    478.100006      3.200000             NaN   
151 2021-06-01           0.6    260.600006      1.700000             NaN   
152 2021-06-02           0.1     96.000000      3.700000             NaN   
153 2021-06-03           0.1     44.599998      2.300000             NaN   
154 2021-06-04           0.0     16.000000      4.700000             NaN   
155 2021-06-05           0.0      9.900000      4.700000             NaN   
156 2021-06-06           0.0     14.500000      4.400000             NaN   
157 2021-06-07           0.0     13.200000      3.800000             NaN   
158 2021-06-08           0.0      0.600000      3.200000             NaN   
159 2021-06-

In [15]:
fs = project.get_feature_store() 

In [16]:
df_wether.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1828 entries, 0 to 1827
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 1828 non-null   datetime64[ns]
 1   temperature_2m_mean  1828 non-null   float32       
 2   precipitation_sum    1828 non-null   float32       
 3   dew_point_2m_mean    1828 non-null   float32       
 4   wind_speed_10m_mean  1828 non-null   float32       
 5   sunshine_duration    1828 non-null   float32       
 6   doy                  1828 non-null   int32         
 7   temp_7d_mean         1828 non-null   float64       
 8   temp_14d_mean        1828 non-null   float64       
 9   precip_3d_sum        1828 non-null   float64       
 10  precip_7d_sum        1828 non-null   float64       
 11  dew_point_3d_mean    1828 non-null   float64       
dtypes: datetime64[ns](1), float32(5), float64(5), int32(1)
memory usage: 128.7 KB


In [17]:
df_pollen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1828 entries, 0 to 1827
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 1828 non-null   datetime64[ns]
 1   alder_pollen         1323 non-null   float32       
 2   birch_pollen         1298 non-null   float32       
 3   grass_pollen         1391 non-null   float32       
 4   mugwort_pollen       1231 non-null   float32       
 5   olive_pollen         1323 non-null   float32       
 6   ragweed_pollen       1390 non-null   float32       
 7   alder_pollen_lag1    1827 non-null   float32       
 8   alder_pollen_lag2    1826 non-null   float32       
 9   alder_pollen_lag3    1825 non-null   float32       
 10  birch_pollen_lag1    1827 non-null   float32       
 11  birch_pollen_lag2    1826 non-null   float32       
 12  birch_pollen_lag3    1825 non-null   float32       
 13  grass_pollen_lag1    1827 non-nul

In [19]:
#Creating the feature groups:
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description = "Daily wether in Stockholm",
    version=5,
    primary_key = ["date"],
    event_time="date",
    time_travel_format="HUDI",
)
#Insert:
weather_fg.insert(df_wether, wait=True)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1286344/fs/1273966/fg/1880583


Uploading Dataframe: 100.00% |██████████| Rows 1828/1828 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_5_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286344/jobs/named/weather_5_offline_fg_materialization/executions
2026-01-02 17:06:13,038 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2026-01-02 17:06:19,606 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-02 17:08:02,617 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2026-01-02 17:08:02,812 INFO: Waiting for log aggregation to finish.
2026-01-02 17:08:11,552 INFO: Execution finished successfully.


(Job('weather_5_offline_fg_materialization', 'SPARK'), None)

In [20]:
pollen_fg = fs.get_or_create_feature_group(
    name='pollen',
    description = "The maximum recorded pollen in Stockholm for various types",
    version=5,
    primary_key = ["date"],
    event_time="date",
    time_travel_format="HUDI",
)
#insert:
pollen_fg.insert(df_pollen, wait=True)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1286344/fs/1273966/fg/1880584


Uploading Dataframe: 100.00% |██████████| Rows 1828/1828 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: pollen_5_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286344/jobs/named/pollen_5_offline_fg_materialization/executions
2026-01-02 17:09:39,477 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2026-01-02 17:09:45,853 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-02 17:11:37,182 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2026-01-02 17:11:37,353 INFO: Waiting for log aggregation to finish.
2026-01-02 17:11:46,293 INFO: Execution finished successfully.


(Job('pollen_5_offline_fg_materialization', 'SPARK'), None)